<h3> [동적 웹크롤링 실습] </h3>
<h4> 무신사 베스트 상품 페이지 상품 정보 웹크롤링 </h4>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# 크롬 드라이버 생성
driver = webdriver.Chrome()

# 페이지 이동
driver.get("https://www.musinsa.com/main/musinsa/ranking?storeCode=musinsa&sectionId=199&categoryCode=000")

# 팝업 닫기
driver.find_element(By.CSS_SELECTOR, "body > section > div._gateway-additional_3fnt1_52 > button").click()
time.sleep(1)

# 스크롤
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
  time.sleep(1)
  new_height = driver.execute_script("return document.body.scrollHeight")
  if last_height == new_height:
    break
  last_height = new_height

# 상품 정보 크롤링
from bs4 import BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
items = soup.select('.sc-1m4cyao-0')
data = []
for item in items:
  brand = item.select_one('.sc-1m4cyao-10.PxNii > a:nth-of-type(odd) > p').text
  name = item.select_one('.sc-1m4cyao-10.PxNii > a:nth-of-type(even) > p').text.strip()
  price = item.select_one('.sc-1m4cyao-11.fulxlo > span:not(.text-red)').text.replace('원', '')
  link = item.select_one('.sc-1m4cyao-10.PxNii > a:nth-of-type(2)').attrs['href']
  print(brand, name, price, link)
  data.append([brand, name, price, link])

In [ ]:
df = pd.DataFrame(data, columns=['브랜드', '상품명', '가격', '상품 링크'])
df

In [35]:
df.to_excel('musinsa_product_rank.xlsx', index = False)